In [22]:
import os
import json
import requests
import openai
from dotenv import load_dotenv
from pprint import pprint
from datetime import datetime
from zoneinfo import ZoneInfo 
import time

In [23]:
# Load environment variables from the .env file
load_dotenv()

True

In [26]:
class GoogleMapsClient:
    """
    A client to interact with various Google Maps Platform APIs.
    """
    GEOCODE_API_URL = "https://maps.googleapis.com/maps/api/geocode/json"
    ELEVATION_API_URL = "https://maps.googleapis.com/maps/api/elevation/json"
    TIMEZONE_API_URL = "https://maps.googleapis.com/maps/api/timezone/json"
    WEATHER_API_URL = "https://weather.googleapis.com/v1"

    def __init__(self, api_key):
        """
        Initializes the client with a Google Maps API Key.
        """
        if not api_key:
            raise ValueError("Google Maps API Key not found. Ensure your .env file is set up correctly.")
        self.api_key = api_key

    def _make_request(self, url, params):
        """
        Internal method to perform API requests, handle errors, and return JSON.
        """
        params['key'] = self.api_key
        try:
            response = requests.get(url, params=params)
            response.raise_for_status()  # Raise an exception for HTTP error codes
            data = response.json()
            if 'error' in data:
                print(f"API Error: {data['error']['message']}")
                return None
            if 'status' in data and data['status'] != 'OK':
                print(f"API Error: {data['status']} - {data.get('error_message', '')}")
                return None
            return data
        except requests.exceptions.RequestException as e:
            print(f"Connection Error: {e}")
            return None
        except ValueError:
             print("Error: A valid JSON response was not received. Response received:")
             print(response.text)
             return None

    def geocode_by_address(self, address, language='en'):
        """
        Gets geolocation data from a text-based address.
        """
        params = {'address': address, 'language': language}
        return self._make_request(self.GEOCODE_API_URL, params)

    def reverse_geocode(self, lat, lng, language='en'):
        """
        Gets geolocation data (reverse geocoding) from coordinates.
        """
        params = {'latlng': f"{lat},{lng}", 'language': language}
        return self._make_request(self.GEOCODE_API_URL, params)

    def get_elevation(self, lat, lng):
        """
        Gets the elevation for a pair of coordinates.
        """
        params = {'locations': f"{lat},{lng}"}
        return self._make_request(self.ELEVATION_API_URL, params)

    def get_timezone(self, lat, lng):
        """
        Gets the time zone information for a pair of coordinates.
        """
        params = {
            'location': f"{lat},{lng}",
            'timestamp': int(time.time()) 
        }
        return self._make_request(self.TIMEZONE_API_URL, params)

    def get_current_conditions(self, lat, lng, units='IMPERIAL'):
        """
        Gets the current weather conditions.
        Units can be 'IMPERIAL' or 'METRIC'.
        """
        url = f"{self.WEATHER_API_URL}/currentConditions:lookup"
        params = {'location.latitude': lat, 'location.longitude': lng, 'unitsSystem': units}
        return self._make_request(url, params)

    def get_daily_forecast(self, lat, lng, units='IMPERIAL', days=None):
        """
        Gets the daily weather forecast.
        Can specify the number of days (e.g., days=6).
        """
        url = f"{self.WEATHER_API_URL}/forecast/days:lookup"
        params = {'location.latitude': lat, 'location.longitude': lng, 'unitsSystem': units}
        # Add the 'days' parameter only if it's provided
        if days is not None:
            params['days'] = days
        return self._make_request(url, params)

    def get_hourly_forecast(self, lat, lng, units='IMPERIAL', hours=None):
        """
        Gets the hourly weather forecast.
        Can specify the number of hours (e.g., hours=25).
        """
        url = f"{self.WEATHER_API_URL}/forecast/hours:lookup"
        params = {'location.latitude': lat, 'location.longitude': lng, 'unitsSystem': units}
        # Add the 'hours' parameter only if it's provided
        if hours is not None:
            params['hours'] = hours
        return self._make_request(url, params)

    def get_hourly_history(self, lat, lng, units='IMPERIAL', hours=None):
        """
        Gets the hourly weather history.
        Can specify the number of hours (e.g., hours=3).
        """
        url = f"{self.WEATHER_API_URL}/history/hours:lookup"
        params = {'location.latitude': lat, 'location.longitude': lng, 'unitsSystem': units}
        # Add the 'hours' parameter only if it's provided
        if hours is not None:
            params['hours'] = hours
        return self._make_request(url, params)


In [27]:
def extract_locations(user_input, api_key):
    """
    Uses OpenAI to extract and consolidate location names from a user's natural language input.
    """
    try:
        client = openai.OpenAI(api_key=api_key)
        prompt = f"""
        You are an expert geographer at identifying and consolidating location information from text.
        Your task is to extract locations and combine them into the most specific strings possible
        for geocoding. If a specific place (like a building, park, or address) is mentioned
        with its city or region, you MUST combine them into a single string. Do not split
        a single conceptual place into multiple parts.

        Your answer MUST be a JSON object with a single key named "result", which contains an
        array of the final location strings.

        Example 1:
        - User query: 'What is the weather forecast for the area around the Northeast Medical Building in Tuscaloosa?'
        - Correct output: {{"result": ["Northeast Medical Building, Tuscaloosa"]}}

        Example 2:
        - User query: 'I want to know the elevation of the Eiffel Tower and the weather in Rome.'
        - Correct output: {{"result": ["Eiffel Tower, Paris", "Rome"]}}

        Now, process the following query:
        User query: '{user_input}'
        """

        response = client.chat.completions.create(
            model="gpt-4o",
            messages=[
                {"role": "system", "content": "You are a helpful and precise location extraction assistant that consolidates location information."},
                {"role": "user", "content": prompt}
            ],
            response_format={"type": "json_object"},
        )

        content = response.choices[0].message.content
        locations = json.loads(content)
        return locations
    except openai.APIError as e:
        print(f"OpenAI API Error: {e}")
        return None
    except json.JSONDecodeError:
        print(f"Error: OpenAI did not return valid JSON. Response was: {content}")
        return None
    except Exception as e:
        print(f"An unexpected error occurred with OpenAI: {e}")
        return None

In [28]:
def convert_and_print_time(utc_time_str, local_tz):
    """Helper function to convert a UTC string to local time and print both."""
    if not utc_time_str or not local_tz:
        return
    try:
        # The 'Z' at the end of the API timestamp means UTC.
        utc_dt = datetime.fromisoformat(utc_time_str.replace('Z', '+00:00'))
        local_dt = utc_dt.astimezone(local_tz)
        print(f"  - API Timestamp (UTC): {utc_dt.strftime('%Y-%m-%d %H:%M:%S %Z')}")
        print(f"  - Converted Local Time: {local_dt.strftime('%Y-%m-%d %H:%M:%S %Z')}")
    except (ValueError, TypeError):
        print("  - Could not parse timestamp.")

In [30]:
def main():
    """
    Main function to run the proof-of-concept workflow.
    """
    google_api_key = os.getenv("GOOGLE_MAPS_API_KEY")
    openai_api_key = os.getenv("OPENAI_API_KEY")

    if not openai_api_key:
        print("OpenAI API Key not found. Exiting.")
        return

    try:
        maps_client = GoogleMapsClient(api_key=google_api_key)
    except ValueError as e:
        print(e)
        return

    user_query = "What is the weather forecast for the area around the Northeast Medical Building in Tuscaloosa and what was it like yesterday?"
    print(f"Processing user query: '{user_query}'\n")

    locations = extract_locations(user_query, openai_api_key)
    if not locations or 'result' not in locations or not locations['result']:
        print("No locations were identified in the user query.")
        return
    print(f"Locations found: {locations['result']}\n")

    for location_name in locations['result']:
        print(f"--- Processing all data for: {location_name} ---")
        
        geo_data = maps_client.geocode_by_address(location_name)
        if not geo_data or not geo_data.get('results'):
            print(f"Could not geocode '{location_name}'. Moving to next location.\n")
            continue
            
        first_result = geo_data['results'][0]
        lat = first_result['geometry']['location']['lat']
        lng = first_result['geometry']['location']['lng']
        print(f"Coordinates found: Lat={lat}, Lng={lng}")

        print("\nFetching Time Zone...")
        local_tz = None
        tz_data = maps_client.get_timezone(lat, lng)
        if tz_data and tz_data.get('timeZoneId'):
            time_zone_id = tz_data['timeZoneId']
            local_tz = ZoneInfo(time_zone_id)
            print(f"Time Zone found: {time_zone_id}")
        else:
            print("Could not retrieve time zone. Weather times will remain in UTC.")

        print("\nFetching Elevation...")
        elevation_data = maps_client.get_elevation(lat, lng)
        pprint(elevation_data)

        print("\nFetching Current Weather...")
        current_weather = maps_client.get_current_conditions(lat, lng)
        if current_weather:
            convert_and_print_time(current_weather.get('currentTime'), local_tz)
            pprint(current_weather)

        print("\nFetching Hourly Forecast...")
        hourly_forecast = maps_client.get_hourly_forecast(lat, lng, hours=24) # Up to 240 hours
        if hourly_forecast and hourly_forecast.get('forecastHours'):
            first_hour = hourly_forecast['forecastHours'][0]
            print("Time conversion for the first hour of the forecast:")
            timestamp = first_hour.get('interval', {}).get('startTime')
            convert_and_print_time(timestamp, local_tz)
            pprint(hourly_forecast)
        
        print("\nFetching Daily Forecast...")
        daily_forecast = maps_client.get_daily_forecast(lat, lng, days=3) # Up to 10 days
        if daily_forecast and daily_forecast.get('forecastDays'):
            first_day = daily_forecast['forecastDays'][0]
            print("Time conversion for the first day's sunrise:")
            timestamp = first_day.get('sunEvents', {}).get('sunriseTime')
            convert_and_print_time(timestamp, local_tz)
            pprint(daily_forecast)

        print("\nFetching Hourly History...")
        history = maps_client.get_hourly_history(lat, lng, hours=3) # Up to 24 hours
        if history and history.get('historyHours'):
            first_history_hour = history['historyHours'][0]
            print("Time conversion for the first hour of the history:")
            timestamp = first_history_hour.get('interval', {}).get('startTime')
            convert_and_print_time(timestamp, local_tz)
            pprint(history)

        print("-" * 50 + "\n")


In [31]:
if __name__ == "__main__":
    main()

Processing user query: 'What is the weather forecast for the area around the Northeast Medical Building in Tuscaloosa and what was it like yesterday?'

Locations found: ['Northeast Medical Building, Tuscaloosa']

--- Processing all data for: Northeast Medical Building, Tuscaloosa ---
Coordinates found: Lat=33.217862, Lng=-87.5335157

Fetching Time Zone...
API Error: REQUEST_DENIED - 
Could not retrieve time zone. Weather times will remain in UTC.

Fetching Elevation...
{'results': [{'elevation': 68.93549346923828,
              'location': {'lat': 33.217862, 'lng': -87.5335157},
              'resolution': 9.543951988220215}],
 'status': 'OK'}

Fetching Current Weather...
{'airPressure': {'meanSeaLevelMillibars': 1019.53},
 'cloudCover': 0,
 'currentConditionsHistory': {'maxTemperature': {'degrees': 84.3,
                                                 'unit': 'FAHRENHEIT'},
                              'minTemperature': {'degrees': 53.7,
                                             